In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import nbodykit.lab
from nbodykit.lab import *
from nbodykit import style, setup_logging

import numpy as np
import MAS_library as MASL
import Pk_library as PKL
import matplotlib.pyplot as plt
plt.style.use(style.notebook)

import sys, pickle, time, os

# nbodykit tool to read custom (non-standard) catalogue file type
from nbodykit.io.base import FileType
# nbodykit tool to creae custom subclass od CatalogSource
from nbodykit.source.catalog.file import FileCatalogFactory
# nbodykit cosmology parameters initialization
from nbodykit.lab import cosmology
from readfof import FoF_catalog
from torch import flatten, from_numpy

setup_logging()

In [2]:
COSMOPAR = {
#                   | Om   | Ob   |   h   |  n_s  | s_8 | Mnu | w |

    'fiducial' :    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'zeldovich':    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Mnu_p' :       [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.1, -1],
    'Mnu_pp' :      [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.2, -1],
    'Mnu_ppp' :     [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.4, -1],
    
    'h_m' :         [0.3175, 0.049, 0.6511, 0.9624, 0.834, 0, -1],
    'h_p' :         [0.3175, 0.049, 0.6911, 0.9624, 0.834, 0, -1],
    
    'ns_m' :        [0.3175, 0.049, 0.6711, 0.9424, 0.834, 0, -1],
    'ns_p' :        [0.3175, 0.049, 0.6711, 0.9824, 0.834, 0, -1],
    
    'Ob_m' :        [0.3175, 0.048, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob_p' :        [0.3175, 0.050, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_m' :       [0.3175, 0.047, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_p' :       [0.3175, 0.051, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Om_m' :        [0.3075, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'Om_p' :        [0.3275, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    's8_m' :        [0.3175, 0.049, 0.6711, 0.9624, 0.819, 0, -1],
    's8_p' :        [0.3175, 0.049, 0.6711, 0.9624, 0.849, 0, -1],
    
    'w_m' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -0.95],
    'w_p' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1.05]
}

fiducial_vals = {
    'Ob'  : 0.3175,
    'Ob2' : 0.3175,
    'Om'  : 0.049,
    'h'   : 0.6711,
    'n_s' : 0.9624,
    'ns'  : 0.9624,
    's_8' : 0.834,
    's8'  : 0.834,
    'Mnu' : 0,
    'w'   : -1
}

def info_name(name):
    """Obtain realization information from namefile"""
    # assert type(name) == str
    print("TYPE: ", type(name))
    # info = name.split('_')
    info = name.split('_')[-3:]
    print("TYPE: ", info, "\n", type(info))
    info[2] = info[2].replace(".wst", "")
    N_hgrid = info[0]
    N_WSTgrid = info[1]
    n_realiz = info[2].replace(".wst", "")
    
    return [int(N_hgrid), int(N_WSTgrid), int(n_realiz)]

def cosmo_parser(name):
    """Obtain cosmology from .wst file"""
    info = name.split('_')
    if info[0] == "fiducial":
        return info[0]
    elif info[0] == "zeldovich":
        return info[0]
    else:
        return info[0] + "_" + info[1]
    
def PacMan(x, d = np.array((0, 0, 1000)) ):
    """Returns a number x in the interval [0; d]"""
    if 0 <= x[2] <= d[2]:
        return x
    elif x[2] > d[2]:
        return PacMan(x - d)
    elif x[2] < 0:
        return PacMan(x + d) 

In [8]:
N_grid = 256
BoxDim = 1000.
line_of_sight = [0, 0, 1]

snapnum = 2
filename = "fiducial"+'_coefficients_' +"08" + '_' + "08" + '_' +str(N_grid)+"_"+str(1000)+'.wst'
snapdir = "/media/fuffolo97/HDD1/UNI/Tesi/halos/" + cosmo_parser(filename) + "/1"

z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[snapnum]
strt = time.time()
H_0 = 100 * COSMOPAR["fiducial"][2]

datas = FoF_catalog(snapdir, snapnum, read_IDs=False)

pos_h = (datas.GroupPos/1e3).astype(np.float32)                     # positions in Mpc/h
mass = (datas.GroupMass * 1e10).astype(np.float32)                  # masses in M_sun/
vel = (datas.GroupVel*(1.0+redshift)).astype(np.float32)            # Halo peculiar velocities in km/s

pos_rsd = []
for i in range(len(vel)):
    pos_rsd.append(PacMan(pos_h[i] + (1+redshift) * np.array( ([0, 0, vel[i][2]/H_0]) ) ))
pos_rsd = np.array(pos_rsd, dtype=np.float32)

In [9]:
assert np.shape(pos_h) == np.shape(pos_rsd) == np.shape(vel)
assert len(pos_h) == len(mass)

In [10]:
dtype_cust = [("Position", (np.float32, 3)), ("RSDPosition", (np.float32, 3)), ("Velocity", (np.float32, 3)), ("Mass", np.float32)]

In [11]:
if os.path.exists('./data_source_fiducial_1.dat'):
    os.remove('./data_source_fiducial_1.dat')

with open('./data_source_fiducial_1.dat', "wb") as ff:
    pos_h.tofile(ff)
    pos_rsd.tofile(ff)
    vel.tofile(ff)
    mass.tofile(ff)
    ff.seek(0)

In [12]:
binCat = BinaryCatalog(ff.name, dtype_cust)

[ 000091.00 ]   0: 09-27 14:58  CatalogSource   INFO     Extra arguments to FileType: ([('Position', (<class 'numpy.float32'>, 3)), ('RSDPosition', (<class 'numpy.float32'>, 3)), ('Velocity', (<class 'numpy.float32'>, 3)), ('Mass', <class 'numpy.float32'>)],) {}


In [13]:
cosmo_nbk = cosmology.Cosmology(h=fiducial_vals["h"],
                                sigma8=fiducial_vals["s8"],
                                #Omega0_b=fiducial_vals["Ob"],
                                Omega0_cdm = fiducial_vals["Om"],
                                n_s=fiducial_vals["ns"],
                                # m_ncdm=fiducial_vals["Mnu"]
                                )

In [14]:
mesh = binCat.to_mesh(resampler='cic', Nmesh=256, compensated=True,
                      position='Position', weight="Mass",
                      BoxSize=BoxDim
                      )

In [15]:
r_3d = FFTPower(mesh, mode='1d', dk=0.005, kmin=0.01)

[ 000162.35 ]   0: 09-27 14:59  CatalogMesh     INFO     Chunk 0 ~ 4194304 / 195784 
[ 000162.65 ]   0: 09-27 14:59  CatalogMesh     INFO     painted 195784 out of 195784 objects to mesh
[ 000162.65 ]   0: 09-27 14:59  CatalogMesh     INFO     painted 195784 out of 195784 objects to mesh
[ 000162.65 ]   0: 09-27 14:59  CatalogMesh     INFO     mean particles per cell is 3.54265e+11
[ 000162.66 ]   0: 09-27 14:59  CatalogMesh     INFO     sum is 5.94357e+18 
[ 000162.67 ]   0: 09-27 14:59  CatalogMesh     INFO     normalized the convention to 1 + delta
[ 000162.90 ]   0: 09-27 14:59  CatalogMesh     INFO     field: (BinaryCatalog(size=195784, FileStack(BinaryFile(path=/home/fuffolo97/TESI/data_source_fiducial_1.dat, dataset=*, ncolumns=4, shape=(195784,)>, ... 1 files)) as CatalogMesh) painting done
